# Task
Mount Google Drive, load the dataset "task1" from the "english" directory, and filter it to include only English language data.

## Mount google drive

### Subtask:
Mount Google Drive to access files stored there.


**Reasoning**:
Mount Google Drive to access files stored there.



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
train_file_path = "/content/drive/MyDrive/Mestrado/Modelos de Linguagem/Trabalho LLM/Tarefa Deteccao semeval 2024-8/SubtaskA-20251024T182540Z-1-001/SubtaskA/subtaskA_train_monolingual.jsonl"
train = pd.read_json(train_file_path, lines=True)
display(train.head())

,text,label,model,source,id
0,Forza Motorsport is a popular racing game that...,1,chatGPT,wikihow,0
1,Buying Virtual Console games for your Nintendo...,1,chatGPT,wikihow,1
2,Windows NT 4.0 was a popular operating system ...,1,chatGPT,wikihow,2
3,How to Make Perfume\n\nPerfume is a great way ...,1,chatGPT,wikihow,3
4,How to Convert Song Lyrics to a Song'\n\nConve...,1,chatGPT,wikihow,4


In [3]:
train.model.value_counts()

,count
model,
human,63351
davinci,14343
chatGPT,14339
dolly,14046
cohere,13678


In [4]:
train.iloc[0]

,0
text,Forza Motorsport is a popular racing game that...
label,1
model,chatGPT
source,wikihow
id,0


In [5]:
train.iloc[0].text

'Forza Motorsport is a popular racing game that provides players with the ability to race on various tracks and in different vehicles. Whether you\'re a seasoned racer or a newbie, playing Forza Motorsport can be a fun experience. In this article, we will take you through the different steps on how to play Forza Motorsport.\n\nStep 1. Insert The Game Disc\n\nThe first step is to insert the game disc into your console or computer. Follow the instructions to set up the game.\n\nStep 2. Choose Your Game\n\nOnce the game is set up, choose the game you\'d like to play. Forza Motorsport has different modes: Career, Free Play, and Arcade. In this article, we will focus on the Arcade mode.\n\nStep 3. Just Make A Quick Race By The Arcade Mode\n\nOnce the Arcade mode is selected, choose "Quick Race" to get started quickly.\n\nStep 4. Pick A Racetrack\n\nPick a racetrack from the different ones available like Road Atlanta, New York, Rio de Janeiro, Maple Valley, or anything to choose from.\n\nSte

In [6]:
import pandas as pd
dev_file_path = "/content/drive/MyDrive/Mestrado/Modelos de Linguagem/Trabalho LLM/Tarefa Deteccao semeval 2024-8/SubtaskA-20251024T182540Z-1-001/SubtaskA/subtaskA_dev_monolingual.jsonl"
dev = pd.read_json(dev_file_path, lines=True)
display(dev.head())

,text,label,model,source,id
0,Giving gifts should always be enjoyable. Howe...,1,bloomz,wikihow,0
1,Yveltal (Japanese: ユベルタル) is one of the main a...,1,bloomz,wikihow,1
2,If you'd rather not annoy others by being rude...,1,bloomz,wikihow,2
3,If you're interested in visiting gravesite(s) ...,1,bloomz,wikihow,3
4,The following are some tips for becoming succe...,1,bloomz,wikihow,4


In [7]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model
from sklearn.metrics import classification_report

In [8]:
# ============================================
# 0️⃣ LOAD DATA (Assumindo que df_sub e dev_sub estão carregados)
# ============================================
# Renomeando para clareza na nova tarefa
train_file_path = "/content/drive/MyDrive/Mestrado/Modelos de Linguagem/Trabalho LLM/Tarefa Deteccao semeval 2024-8/SubtaskA-20251024T182540Z-1-001/SubtaskA/subtaskA_train_monolingual.jsonl"
df_train = pd.read_json(train_file_path, lines=True)
dev_file_path = "/content/drive/MyDrive/Mestrado/Modelos de Linguagem/Trabalho LLM/Tarefa Deteccao semeval 2024-8/SubtaskA-20251024T182540Z-1-001/SubtaskA/subtaskA_dev_monolingual.jsonl"
df_dev = pd.read_json(dev_file_path, lines=True)

# ============================================
# 1️⃣ PREPROCESS DATA (Adaptado para Human vs. AI)
# ============================================

# Função para criar o rótulo binário
def map_label(model_name):
    if model_name == 'human':
        return 'human'
    else:
        return 'AI' # Agrupa 'chatGPT', 'davinci', etc.

# Aplica a função para criar uma nova coluna de rótulo legível
df_train['label_str'] = df_train['model'].apply(map_label)
df_dev['label_str'] = df_dev['model'].apply(map_label)

# Mapeia os rótulos de string para inteiros
label_map = {"human": 0, "AI": 1}
df_train["label_id"] = df_train["label_str"].map(label_map)
df_dev["label_id"] = df_dev["label_str"].map(label_map)

print("Label distribution (train):")
print(df_train["label_str"].value_counts())
print("\nLabel distribution (dev):")
print(df_dev["label_str"].value_counts())

# ============================================
# 2️⃣ DEFINE DATASET CLASS (Adaptado)
# ============================================

# class TextDataset(Dataset):
#     def __init__(self, df, tokenizer, max_len=128):
#         # MUDANÇA: "sentence" -> "text"
#         self.texts = df["text"].tolist()
#         # MUDANÇA: "label" -> "label_id" (nossa nova coluna)
#         self.labels = df["label_id"].tolist()
#         self.tokenizer = tokenizer
#         self.max_len = max_len

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         text = str(self.texts[idx]) # Adicionado str() para segurança
#         label = self.labels[idx]
#         enc = self.tokenizer(
#             text,
#             truncation=True,
#             padding="max_length",
#             max_length=self.max_len,
#             return_tensors="pt",
#         )
#         item = {key: val.squeeze() for key, val in enc.items()}
#         item["labels"] = torch.tensor(label, dtype=torch.long)
#         return item

class TextDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128):
        self.texts = df["text"].tolist()
        self.labels = df["label_id"].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        # MUDANÇA: Removido padding, return_tensors.
        # Deixe o DataCollator do Trainer cuidar disso.
        enc = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_len,
            # Não adicione padding aqui!
        )

        # Retorne um dicionário simples.
        # O Trainer cuidará de agrupar e converter para tensores.
        item = {
            "input_ids": enc["input_ids"],
            "attention_mask": enc["attention_mask"],
            "labels": label
        }
        return item

# ============================================
# 3️⃣ LOAD PRETRAINED MODEL + TOKENIZER (Adaptado)
# ============================================

model_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2, # Ainda é classificação binária
    # MUDANÇA: Atualiza os rótulos para a nova tarefa
    id2label={0: "human", 1: "AI"},
    label2id={"human": 0, "AI": 1}
)

# ============================================
# 4️⃣ ADD LORA ADAPTER (Sem mudanças)
# ============================================

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS",
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# ============================================
# 5️⃣ CREATE DATASETS (Adaptado)
# ============================================

# Passa os DataFrames corretos
train_dataset = TextDataset(df_train, tokenizer)
dev_dataset = TextDataset(df_dev, tokenizer)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Dev dataset size: {len(dev_dataset)}")

# ============================================
# 6️⃣ DEFINE METRICS (Sem mudanças)
# ============================================

def compute_metrics(pred):
    preds = np.argmax(pred.predictions, axis=1)
    labels = pred.label_ids
    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

# ============================================
# 7️⃣ TRAINING ARGUMENTS (Sem mudanças, mantendo a correção do log)
# ============================================
# ============================================
# 7️⃣ TRAINING ARGUMENTS (Adaptado para dataset GRANDE)
# ============================================

# Calcule o total de passos de treino (ex: para 1 época com batch 32)
# 119757 / 32 = 3742.4 => 3743 passos
# Vamos definir nossos logs/evals com base nisso. ex: a cada 1000 passos.

training_args = TrainingArguments(
    output_dir="./lora_results_human_ai",
    dataloader_num_workers=2,
    # --- Mudanças Estratégicas ---
    num_train_epochs=1, # 120k amostras é muito. 1 época é um ótimo ponto de partida.

    # Aumente o batch size para acelerar o treino (depende da sua VRAM)
    per_device_train_batch_size=16,  # Era 16. Tente 128 ou 128.
    per_device_eval_batch_size=16,   # Pode ser igual ou maior que o de treino.

    # Mude TUDO de "epoch" para "steps"
    eval_strategy="steps",
    logging_strategy="steps",
    save_strategy="steps",

    # Defina a frequência em passos
    eval_steps=1000,     # Avalia no dataset de dev a cada 1000 passos
    logging_steps=1000,  # Mostra o training loss a cada 1000 passos
    save_steps=1000,     # Salva um checkpoint a cada 1000 passos

    # --- O resto pode ficar igual ---
    learning_rate=2e-4,
    weight_decay=0.01,
    logging_dir="./logs_human_ai",
    load_best_model_at_end=True, # ESSENCIAL: vai carregar o melhor checkpoint (ex: passo 3000) no final
    metric_for_best_model="accuracy",
    report_to="none",
)

# =G==========================================
# 8️⃣ TRAINER SETUP (Sem mudanças)
# ============================================

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator, # <-- 3. PASSE-O PARA O TRAINER
)

# ============================================
# 9️⃣ TRAIN MODEL (Sem mudanças)
# ============================================

trainer.train()

# ============================================
# 🔟 EVALUATE MODEL (Adaptado)
# ============================================

preds_output = trainer.predict(dev_dataset)
y_true = preds_output.label_ids
y_pred = np.argmax(preds_output.predictions, axis=1)

# MUDANÇA: Atualiza o mapa inverso
inv_label_map = {0: "human", 1: "AI"}
y_true_labels = [inv_label_map[i] for i in y_true]
y_pred_labels = [inv_label_map[i] for i in y_pred]

print("\n--- Classification Report (Human vs. AI) ---")
print(classification_report(y_true_labels, y_pred_labels))

Label distribution (train):
label_str
human    63351
AI       56406
Name: count, dtype: int64

Label distribution (dev):
label_str
AI       2500
human    2500
Name: count, dtype: int64


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 296,450 || all params: 109,780,228 || trainable%: 0.2700
Train dataset size: 119757
Dev dataset size: 5000


Step,Training Loss,Validation Loss,Accuracy
1000,0.371200,1.010674,0.636200
2000,0.259600,0.964533,0.661600
3000,0.245700,0.959750,0.680400
4000,0.221200,0.910559,0.688400
5000,0.213200,0.845860,0.688000
6000,0.205100,0.834596,0.710800
7000,0.201000,0.798629,0.715800



--- Classification Report (Human vs. AI) ---
              precision    recall  f1-score   support

          AI       0.72      0.71      0.71      2500
       human       0.71      0.72      0.72      2500

    accuracy                           0.72      5000
   macro avg       0.72      0.72      0.72      5000
weighted avg       0.72      0.72      0.72      5000



In [11]:
from google.colab import drive
import os

# --- 2. Defina um caminho PERMANENTE ---
# Crie um diretório dentro do seu Drive para este modelo
output_dir = "/content/drive/MyDrive/Mestrado/Modelos de Linguagem/Trabalho LLM/Tarefa Deteccao semeval 2024-8/human_ai_lora_final"

# Crie o diretório se ele não existir
os.makedirs(output_dir, exist_ok=True)
print(f"Diretório de salvamento: {output_dir}")

# ============================================
# 11. SALVAR O MODELO FINAL
# ============================================

print(f"Salvando adaptadores LoRA em {output_dir}...")

# 1. Salve os pesos do adaptador PEFT (LoRA)
model.save_pretrained(output_dir)

# 2. Salve o tokenizer
tokenizer.save_pretrained(output_dir)

print(f"Modelo e tokenizer salvos permanentemente em seu Google Drive!")

# Verifique os arquivos salvos no seu Drive
!ls -lh {output_dir}

Diretório de salvamento: /content/drive/MyDrive/Mestrado/Modelos de Linguagem/Trabalho LLM/Tarefa Deteccao semeval 2024-8/human_ai_lora_final
Salvando adaptadores LoRA em /content/drive/MyDrive/Mestrado/Modelos de Linguagem/Trabalho LLM/Tarefa Deteccao semeval 2024-8/human_ai_lora_final...
Modelo e tokenizer salvos permanentemente em seu Google Drive!
ls: cannot access '/content/drive/MyDrive/Mestrado/Modelos': No such file or directory
ls: cannot access 'de': No such file or directory
ls: cannot access 'Linguagem/Trabalho': No such file or directory
ls: cannot access 'LLM/Tarefa': No such file or directory
ls: cannot access 'Deteccao': No such file or directory
ls: cannot access 'semeval': No such file or directory
ls: cannot access '2024-8/human_ai_lora_final': No such file or directory


In [12]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel
from google.colab import drive

# --- 1. Monte seu Google Drive ---
# print("Montando Google Drive...")
# drive.mount('/content/drive')

# ============================================
# 12. CARREGAR O MODELO SALVO DO DRIVE
# ============================================

# --- 2. Defina os caminhos ---
base_model_name = "bert-base-uncased"
# O caminho exato onde você salvou os adaptadores no seu Drive
adapter_dir = "/content/drive/MyDrive/Mestrado/Modelos de Linguagem/Trabalho LLM/Tarefa Deteccao semeval 2024-8/human_ai_lora_final"

print(f"Carregando o modelo base: {base_model_name}...")

# --- 3. Carregue o Modelo Base ---
base_model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    num_labels=2,
    id2label={0: "human", 1: "AI"},
    label2id={"human": 0, "AI": 1}
)

print(f"Carregando e aplicando adaptadores LoRA de: {adapter_dir}...")

# --- 4. Aplique os Adaptadores LoRA ---
model = PeftModel.from_pretrained(base_model, adapter_dir)

# --- 5. Carregue o Tokenizer ---
tokenizer = AutoTokenizer.from_pretrained(adapter_dir)

print("Modelo e tokenizer carregados do Google Drive!")

# --- 6. Prepare o modelo para inferência ---
model.eval()
if torch.cuda.is_available():
    model.to("cuda")
    print("Modelo movido para a GPU.")

Carregando o modelo base: bert-base-uncased...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Carregando e aplicando adaptadores LoRA de: /content/drive/MyDrive/Mestrado/Modelos de Linguagem/Trabalho LLM/Tarefa Deteccao semeval 2024-8/human_ai_lora_final...
Modelo e tokenizer carregados do Google Drive!
Modelo movido para a GPU.
